# ResNet Training

В этом ноутбуке будет произведено обучение модели `Resnet34` на полученном наборе данных из ноутбука
`./notebooks/01. get_train_data`.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!pip install -r drive/MyDrive/detectime/requirements.txt

In [4]:
cd drive/MyDrive/detectime/

/content/drive/MyDrive/detectime


In [5]:
import os
import sys
import logging
import numpy as np
import random
from tqdm import tqdm

sys.path.append('.')
from definitions import ROOT_DIR
from detectime.utils import (
    save_checkpoint,
    train_valid_split
)
from detectime.models import load_model
from detectime.loss_function import get_loss
from detectime.optimizers import (
    get_optimizer,
    get_scheduler
)
from detectime.dataset import get_data_loaders
from detectime.train import train, validation

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-07-07 10:23:08", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://ipykernel.pylab.backend_inline version unknown."}


In [6]:
DATA_PATH = ROOT_DIR / 'data'
NOTEBOOK_PATH = ROOT_DIR / 'notebooks'
INPUT_DATA = DATA_PATH / 'INPUT_DATA'
INPUT_IMAGES_FOLDER = INPUT_DATA / 'TRAIN_DATA'
TRAIN_IMG_FOLDER = INPUT_DATA / 'TRAIN_IMG'
SAVE_TRAIN_IMAGES_HANDS = TRAIN_IMG_FOLDER / 'HANDS'
SAVE_TRAIN_IMAGES_FACES= TRAIN_IMG_FOLDER / 'FACES'

JSON_FOLDER = INPUT_DATA / 'JSON'
FACES_JSON_PRETRAINED = JSON_FOLDER / 'train_with_bboxes.json'
TRAIN_LABELS = INPUT_DATA / 'train.csv'
HAND_DETECTION_FOLDER = ROOT_DIR / 'model' / 'mask_rcnn_hand_detection.h5'
ANNOTATION_DATA = JSON_FOLDER / 'hands.json'

Подгрузим известный нам конфиг и будем использовать `device` для обучения - либо `cpu`, либо `cuda`.

In [19]:
import torch
import yaml
from detectime.utils import convert_dict_to_tuple

CONFIG_PATH = ROOT_DIR / 'config.yml'

with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cuda


Обеспечим консистентность и воспроизводимость системы.

In [8]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

Подгрузим веса модели `ResNet34`.

In [9]:
print("Loading model...")
net = load_model(config, device=device_name)
print("Done.")

Loading model...
{"asctime": "2021-07-07 10:23:11", "name": "detectime.models", "filename": "models.py", "levelname": "INFO", "message": "ResNet34"}
Done.


Подгрузим optimizer, scheduler и loss aggregator.

In [10]:
criterion, criterion_val = get_loss(config, device=device_name)

In [11]:
optimizer = get_optimizer(config, net)

{"asctime": "2021-07-07 10:23:14", "name": "detectime.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "0.002"}
{"asctime": "2021-07-07 10:23:14", "name": "detectime.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "Opt: SGD"}


In [12]:
n_epoch = config.train.n_epoch
scheduler = get_scheduler(config, optimizer)
train_epoch = tqdm(range(config.train.n_epoch),
                   dynamic_ncols=True,
                   desc='Epochs',
                   position=0)

Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

Разделим данные на обучение и валидацию по ключу `video_name`.

In [22]:
train_data, val_data = train_valid_split(
    ANNOTATION_DATA,
    JSON_FOLDER,
    val_size=0.15
)

{"asctime": "2021-07-07 11:05:07", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "3402"}
{"asctime": "2021-07-07 11:05:07", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "132"}
{"asctime": "2021-07-07 11:05:07", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "train data length=3029"}
{"asctime": "2021-07-07 11:05:07", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "validation data length=373"}
{"asctime": "2021-07-07 11:05:07", "name": "detectime.utils", "filename": "utils.py", "levelname": "INFO", "message": "Savedir train and valid data: /content/drive/MyDrive/detectime/data/INPUT_DATA/JSON"}


In [23]:
ANNOTATIONS_TRAIN = JSON_FOLDER / 'train.json'
ANNOTATIONS_VALID = JSON_FOLDER / 'valid.json'

In [24]:
dt, dv = get_data_loaders(config,
                          INPUT_IMAGES_FOLDER,
                          ANNOTATIONS_TRAIN,
                          ANNOTATIONS_VALID
                          )

out_dir = str(INPUT_DATA / os.path.join(config.outdir, config.exp_name))
print("Savedir: {}".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

{"asctime": "2021-07-07 11:05:13", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing train reader..."}
{"asctime": "2021-07-07 11:05:13", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
{"asctime": "2021-07-07 11:05:13", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing valid reader..."}
{"asctime": "2021-07-07 11:05:13", "name": "detectime.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
Savedir: /content/drive/MyDrive/detectime/data/INPUT_DATA/EXPERIMENTS/detectime


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
for epoch in train_epoch:
    train(net, dt, criterion, optimizer, config, epoch)
    validation(net, dv, criterion_val, epoch)
    save_checkpoint(net, optimizer, scheduler, epoch, out_dir)
    scheduler.step()


Train:   0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

Train: 100%|██████████| 63/63 [01:33<00:00,  1.49s/it]


Val:   0%|          | 0/8 [00:00<?,

Train process of epoch: 0 is done; 
 loss: 0.8038; acc: 0.73




Val:  12%|█▎        | 1/8 [00:05<00:36,  5.23s/it]

Val:  25%|██▌       | 2/8 [00:05<00:13,  2.31s/it]

Val:  50%|█████     | 4/8 [00:12<00:11,  2.96s/it]

Val:  62%|██████▎   | 5/8 [00:16<00:09,  3.28s/it]

Val:  75%|███████▌  | 6/8 [00:16<00:04,  2.48s/it]

Val:  88%|████████▊ | 7/8 [00:18<00:02,  2.24s/it]

Val: 100%|██████████| 8/8 [00:18<00:00,  2.37s/it]


Validation of epoch: 0 is done; 
 loss: 0.4213; acc: 0.86


Train: 100%|██████████| 63/63 [01:15<00:00,  1.20s/it]


Val:   0%|          | 0/8 [00:00<?, ?it/s]

...

Train: 100%|██████████| 63/63 [01:14<00:00,  1.19s/it]


Val:   0%|          | 0/8 [00:00<?, ?it/s]

Train process of epoch: 4 is done; 
 loss: 0.0908; acc: 0.99




Val:  12%|█▎        | 1/8 [00:07<00:51,  7.40s/it]

Val:  25%|██▌       | 2/8 [00:07<00:19,  3.19s/it]

Val:  62%|██████▎   | 5/8 [00:07<00:02,  1.07it/s]

Val: 100%|██████████| 8/8 [00:08<00:00,  1.02s/it]


Validation of epoch: 4 is done; 
 loss: 0.1283; acc: 0.96


Train:  25%|██▌       | 16/63 [00:24<00:14,  3.25it/s]